In [6]:
from matplotlib import pyplot as plt
import numpy as np
import meep as mp
import math
import cmath

shape_size = 48
sx, sy, sz = 1, 1, 4
h = 1.25
dpml = 0.5
b_m, c_m = 1.4, 3.54
res = 17
echo = 1000
cell_size = mp.Vector3(sx,sy,sz)
fcen = 0.5
df = 0.2
theta = math.radians(0)
nfreq = 200

# k with correct length (plane of incidence: XZ) 
k = mp.Vector3(math.sin(theta),0,math.cos(theta)).scale(fcen)
def pw_amp(k, x0):
    def _pw_amp(x):
        return cmath.exp(1j * 2 * math.pi * k.dot(x + x0))
    return _pw_amp

def get_spec(batch, index):
    b = np.genfromtxt('data/DATA'+str(batch) +'_sh.txt')
    start = index * shape_size
    xc = b[start:start + shape_size, 0]
    yc = b[start:start + shape_size, 1]
    vertices = [mp.Vector3(xc[0],yc[0])]

    for j in range(1, len(xc) - 1):
        # eliminate duplicate point
        if xc[j] == xc[j - 1] and yc[j] == yc[j - 1]:
            continue
        vertices.append(mp.Vector3(xc[j], yc[j]))
        #print(xc[i][j], yc[i][j])
    # calculate transmission
    bend = get_bend(vertices)
    Ts = []
    st = np.genfromtxt('data/straight.txt')
    for j in range(nfreq):
        Ts = np.append(Ts, bend[j]/st[j])
    return Ts
#     plt.axis('equal')
#     plt.ylim(-0.5, 0.5)
#     plt.xlim(-0.5, 0.5)
#     plt.plot(x, y)
#     plt.fill(x, y)
#     plt.show()
def get_bend(vertices):
    geometry = [mp.Block(size = cell_size, material=mp.Medium(index=b_m)),
                mp.Prism(vertices, 
                         height=h, 
                         material=mp.Medium(index=c_m),
                         center=mp.Vector3()
                        )]
    pml_layers = [mp.PML(thickness=1, direction = mp.Z, side=mp.High),
                  mp.Absorber(thickness=1,direction = mp.Z, side=mp.Low)]
    src_pos = -(sz/2 - dpml - 0.5)
    src = [mp.Source(src = mp.GaussianSource(fcen, fwidth=df),
                     component = mp.Ey,
                     center = mp.Vector3(0,0,src_pos),
                     size = mp.Vector3(sx,sy,0),
                     amp_func=pw_amp(k,mp.Vector3(0,0,src_pos)))]
    sim = mp.Simulation(resolution=res,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        sources=src,
                        geometry=geometry,
                        k_point=k)
    freg = mp.FluxRegion(center=mp.Vector3(0,0,-src_pos),
                         size = mp.Vector3(sx,sy,0))
    trans = sim.add_flux(fcen, df, nfreq, freg)
    sim.run(until = echo)
    bend = mp.get_fluxes(trans)
    return bend

In [13]:
Ts = get_spec(62,186)

-----------
Initializing structure...
Meep: using complex fields.
Meep progress: 12.617647058823529/1000.0 = 1.3% done in 4.0s, 313.5s to go
Meep progress: 25.941176470588236/1000.0 = 2.6% done in 8.0s, 300.7s to go
Meep progress: 39.3235294117647/1000.0 = 3.9% done in 12.0s, 293.6s to go
Meep progress: 52.588235294117645/1000.0 = 5.3% done in 16.0s, 288.6s to go
Meep progress: 65.82352941176471/1000.0 = 6.6% done in 20.0s, 284.2s to go
Meep progress: 79.02941176470588/1000.0 = 7.9% done in 24.0s, 280.0s to go
Meep progress: 92.1470588235294/1000.0 = 9.2% done in 28.0s, 276.2s to go
Meep progress: 105.58823529411765/1000.0 = 10.6% done in 32.0s, 271.5s to go
Meep progress: 119.02941176470588/1000.0 = 11.9% done in 36.0s, 266.8s to go
Meep progress: 132.41176470588235/1000.0 = 13.2% done in 40.1s, 262.4s to go
Meep progress: 145.8235294117647/1000.0 = 14.6% done in 44.1s, 258.1s to go
Meep progress: 159.11764705882354/1000.0 = 15.9% done in 48.1s, 254.1s to go
Meep progress: 172.5294117

In [14]:
Ts = []
st = np.genfromtxt('data/straight.txt')
for j in range(nfreq):
    Ts = np.append(Ts, bend[j]/st[j])

In [15]:
np.savetxt('data/DATA'+str(62)+'_sp186'+'.txt', Ts)